# Extraindo dados da Covid19 via API
### Fonte de Dados: https://brasil.io/dataset/covid19/
### Augusto SPINELLI

### Licença
Os dados dados convertidos estão sob a licença Creative Commons Attribution ShareAlike. Caso utilize os dados, cite a fonte original e quem tratou os dados, como: Fonte: Secretarias de Saúde das Unidades Federativas, dados tratados por Álvaro Justen e colaboradores/Brasil.IO. Caso compartilhe os dados, utilize a mesma licença.

In [1]:
#importando dependencias
import pandas as pd
import numpy as np
import os

#Caminho dos arquivos RAW
raw_data_path = os.path.join(os.path.pardir, 'data', 'raw')
data_path = os.path.join(raw_data_path, 'estados_historico.json')

#criando dataframes brutos/raw
raw_df = pd.read_json(data_path)

#transformando dados
estados = raw_df['results']
df = pd.read_json(estados.to_json(),orient='index')

#verificando informações do dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2928 entries, 0 to 2927
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   city                            0 non-null      float64       
 1   city_ibge_code                  2928 non-null   int64         
 2   confirmed                       2928 non-null   int64         
 3   confirmed_per_100k_inhabitants  2922 non-null   float64       
 4   date                            2928 non-null   datetime64[ns]
 5   death_rate                      2928 non-null   float64       
 6   deaths                          2928 non-null   int64         
 7   estimated_population_2019       2928 non-null   int64         
 8   is_last                         2928 non-null   bool          
 9   order_for_place                 2928 non-null   int64         
 10  place_type                      2928 non-null   object        
 11  stat

## Nightingale Rose Chart ViZ

Gráfico para mostrar evoluções na distribuição. Muito utilizado em jogos de futebol para mostrar estatísticas de gols por temporada

In [19]:
#Primeiro vamos pegar somente os estados do Norte do País
df_norte = df[(df['state'].isin(['AM','RR','RO','AP','PA','TO','AC'])) & (df['date'].isin(['2020-07-01', '2020-06-01','2020-05-01','2020-04-01']))]

### Rosa de Nightingale da região NORTE 

Na rosa de nighlingale do Norte do Brasil notamos que em Abril quase não havia casos nos estados do norte mas nos meses seguintes a pandemia explodiu, com força absurda especialmente no Amazonas e Pará

In [20]:
import plotly.express as px

#setting the parameters of the chart
fig = px.bar_polar(df_norte, r="confirmed", theta="state",  #r is the values, theta= data you wish to compare
                   color="date", template="plotly_dark")  #color is the value of stacked columns 

#adding title, circular grid shape and labels
fig.update_layout(
    title='Brazil North - Evolution of confirmed cases by State',
    template=None,
    polar = dict(gridshape='circular',bgcolor='lightgray',
        radialaxis = dict(range=[0, 160000], ticks='')  #setting the scale
    ))
fig.show()

## Sankey Diagram

Diagrama de Sankey mostra o fluxo de uma quantidade para outra quantidade. É muito utilizado, por exemplo, para visualizar pra onde vai o dinheiro do nosso salário (orçamentos).

O diagrama de sankey abaixo o fluxo de distribuição de mortes por covid por estado

In [38]:
import plotly.graph_objects as go
label = ["AC", "AM", "AP", "PA", "RO", "RR","TO", "Total"]  #total nodes involved in the graph

#deaths number by state (most recent number is max)
am = df_norte[df_norte.state == 'AM'].deaths.max()
ap = df_norte[df_norte.state == 'AP'].deaths.max()
ac = df_norte[df_norte.state == 'AC'].deaths.max()
ro = df_norte[df_norte.state == 'RO'].deaths.max()
rr = df_norte[df_norte.state == 'RR'].deaths.max()
to = df_norte[df_norte.state == 'TO'].deaths.max()
pa = df_norte[df_norte.state == 'PA'].deaths.max()
total = am + ap + ac + ro + rr + to + pa

#creating a sankey diagram using plotly
fig = go.Figure(data=[go.Sankey(       
    node = dict(            #editing properties of the node
      thickness = 15,
      line = dict(color = "black"),
      label = ["AC", "AM", "AP", "PA", "RO", "RR","TO", "Total"],
    ),
    #editing properties of the connecting link
    link = dict(               
      source = [7, 7, 7, 7, 7, 7, 7],  #source nodes
      target = [0, 1, 2, 3, 4, 5, 6],   #target node
      value = [ac, am, ap, pa, ro, rr, to, total],  #value of the links
      color = '#eee0e5'
  ))])

#setting figure title and font style
fig.update_layout(title_text="Distribuição das mortes região Norte", font=dict(size = 12, color = 'maroon'),paper_bgcolor='white')
fig.show()

# End of Notebook